<a href="https://colab.research.google.com/github/Smart-Pigs/Cow_Food_Consumption/blob/main/Data_Cleaning_Two_Cows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
#Change path
import os
path = '/content/gdrive/MyDrive/Cow_eating_label/'
os.chdir(path)

#concatenate for outliers detection 

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
import glob
import numpy as np
import pandas as pd

#remove frist two columns of counts
df = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/uncleaned/Copy of 6051_all.csv')
column_names = df.columns.tolist()
print(column_names)

#prepare dataset
folder = '/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/uncleaned/'
output_df='/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/uncleaned/'
df = pd.DataFrame()
for file in os.listdir(folder):
  if file.endswith(".csv"):
    file_path = os.path.join(folder, file)
    file_df = pd.read_csv(file_path)
    file_df=file_df[column_names]
    file_df["label"] = file_df["label"].replace("grazing", "eating")
    file_df["label"] = file_df["label"].fillna("not_eating")
    file_df["label"] = file_df["label"].replace("ruminating", "not_eating")
    file_df=file_df[~file_df["label"].str.contains("sleeping")]
    df = pd.concat([df, file_df], ignore_index=True)
    print(f'Add {file}')
output = os.path.join(output_df, 'Two_all.csv')
df.to_csv(output, index=False)
    


['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg', 'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar', 'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose', 'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose', 'gyro_z_nose', 'label']
Add Copy of 6051_all.csv
Add Copy of 6065_all.csv


In [45]:
# 1.5IQR
def Detect_outlier_IQR(df):
  Q1 = df.quantile(0.25)
  Q3 = df.quantile(0.75)
  IQR = Q3 - Q1
  down_threshold = Q1-1.5*IQR
  up_threshold = Q3+1.5*IQR
  return up_threshold,down_threshold


#label set
labels=set(df['label'])
df=pd.read_csv('Two_Cows/concatenated/Two_all.csv')
outliers=0
out_percent=0
for label in labels:
  data_item = df[df['label'] == label]
  print(f"Label: {label}")
  for col in data_item.columns[:-1]:
    col_threshold = Detect_outlier_IQR(data_item[col])
    outliers = data_item[(data_item[col]<col_threshold[1])|(data_item[col]>col_threshold[0])].shape[0]
    print(f"{col}: {col_threshold}")
  print("\n")




Label: not_eating
acc_x_leg: (3.3591796875, -2.9189453125)
acc_y_leg: (-9.575097656250001, -10.139746093749999)
acc_z_leg: (0.7955322265625, -1.9320068359375)
gyro_x_leg: (0.8536585122346875, -1.3414633721113203)
gyro_y_leg: (0.9603658318519597, -2.088414609432221)
gyro_z_leg: (2.0884146839380264, -2.423780545592308)
acc_x_maxilar: (11.809765625000002, -1.1005859375000018)
acc_y_maxilar: (12.46114501953125, 2.091711425781251)
acc_z_maxilar: (0.9396850585937502, -8.204748535156252)
gyro_x_maxilar: (55.7774395942688, -57.75914716720581)
gyro_y_maxilar: (33.06402516365051, -32.42378115653992)
gyro_z_maxilar: (31.57012152671814, -32.454267740249634)
acc_x_nose: (7.8135620117187505, -6.297863769531251)
acc_y_nose: (-4.346118164062503, -12.327758789062496)
acc_z_nose: (10.351489257812501, -3.7072998046875005)
gyro_x_nose: (35.152440547943115, -35.823172092437744)
gyro_y_nose: (41.326220989227295, -42.454269886016846)
gyro_z_nose: (29.67987895011902, -29.222562074661255)


Label: eating
acc_x

In [28]:
out_percent=outliers/df.shape[0] 
print(out_percent) 

0.023669895546232417


#Back to original files and remove outliers

In [29]:
#Label: eating
thres_eat={
  'acc_x_leg': (2.8046997070312503, -1.9182495117187501),
  'acc_y_leg': (-9.654650878906251, -10.099670410156248),
  'acc_z_leg': (0.586181640625, -1.8925292968749998),
  'gyro_x_leg': (0.9451219290494915, -1.4939024001359937),
  'gyro_y_leg': (1.0518292486667633, -2.240853637456894),
  'gyro_z_leg': (2.4695122092962265, -2.89634145796299),
  'acc_x_maxilar': (12.869677734375001, 2.734716796875),
  'acc_y_maxilar': (12.035864257812502, -3.0756591796875004),
  'acc_z_maxilar': (1.4158081054687508, -8.130578613281251),
  'gyro_x_maxilar': (74.0396375656128, -74.13110065460205),
  'gyro_y_maxilar': (48.55183172225952, -47.66768503189087),
  'gyro_z_maxilar': (57.95732021331787, -60.823174476623535),
  'acc_x_nose': (7.2267822265625, -6.918139648437501),
  'acc_y_nose': (-3.4417236328125007, -13.5384033203125),
  'acc_z_nose': (12.300244140625002, -7.079638671875002),
  'gyro_x_nose': (61.70731782913208, -60.24390363693237),
  'gyro_y_nose': (53.97865915298462, -55.16768407821655),
  'gyro_z_nose': (43.00304841995239, -42.484755992889404)
}


#Label: not_eating
thres_neat = {
  'acc_x_leg': (3.3591796875, -2.9189453125),
  'acc_y_leg': (-9.575097656250001, -10.139746093749999),
  'acc_z_leg': (0.7955322265625, -1.9320068359375),
  'gyro_x_leg': (0.8536585122346875, -1.3414633721113203),
  'gyro_y_leg': (0.9603658318519597, -2.088414609432221),
  'gyro_z_leg': (2.0884146839380264, -2.423780545592308),
  'acc_x_maxilar': (11.809765625000002, -1.1005859375000018),
  'acc_y_maxilar': (12.46114501953125, 2.091711425781251),
  'acc_z_maxilar': (0.9396850585937502, -8.204748535156252),
  'gyro_x_maxilar': (55.7774395942688, -57.75914716720581),
  'gyro_y_maxilar': (33.06402516365051, -32.42378115653992),
  'gyro_z_maxilar': (31.57012152671814, -32.454267740249634),
  'acc_x_nose': (7.8135620117187505, -6.297863769531251),
  'acc_y_nose': (-4.346118164062503, -12.327758789062496),
  'acc_z_nose': (10.351489257812501, -3.7072998046875005),
  'gyro_x_nose': (35.152440547943115, -35.823172092437744),
  'gyro_y_nose': (41.326220989227295, -42.454269886016846),
  'gyro_z_nose': (29.67987895011902, -29.222562074661255)
}




In [46]:
import glob
import numpy as np
import pandas as pd

Input_path = '/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/uncleaned/'
Output_path = '/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/cleaned/'

df = pd.DataFrame()
for file_name in os.listdir(Input_path):
  if file_name.endswith('.csv'):
    file_path = os.path.join(Input_path, file_name)
    df = pd.read_csv(file_path)
    df = df[df.columns.tolist()]
    df["label"] = df["label"].replace("grazing", "eating")
    df["label"] = df["label"].fillna("not_eating")
    df["label"] = df["label"].replace("ruminating", "not_eating")
    df=df[~df["label"].str.contains("sleeping")]
    for label, threshold in [('eating', thres_eat), ('not_eating', thres_neat)]:
      mask = df['label'] == label
      for col in df.columns[:-1]: 
        if col in threshold:
          # Remove outliers
          upper_thresh, lower_thresh = threshold[col]
          outside_thresh_mask = (df[col] < lower_thresh) | (df[col] > upper_thresh)
          df.loc[outside_thresh_mask & mask, col] = np.nan
          # Interpolate  
          df.loc[mask, col] = df.loc[mask, col].interpolate(method='linear', limit_direction='both')
          # df_new = df_new.append(df)
    clean_file_path = os.path.join(Output_path, f'{file_name}')
    df.to_csv(clean_file_path, index=False)
    print(f'Saved {clean_file_path}')
    df = pd.DataFrame()









Saved /content/gdrive/MyDrive/Cow_eating_label/Two_Cows/cleaned/Copy of 6051_all.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Two_Cows/cleaned/Copy of 6065_all.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Two_Cows/cleaned/Two_all.csv


# Windowing+ Feature extraction for each blocks

In [47]:
import glob
import numpy as np
import pandas as pd

df_train = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/cleaned/Copy of 6051_all.csv')
df_test = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/cleaned/Copy of 6065_all.csv')

In [48]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [49]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [50]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr

In [51]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [52]:
import warnings
warnings.filterwarnings('ignore')

## Test 1&2


In [53]:
dftest_np =df_test.to_numpy()
dftrain_np =df_train.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [54]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(2287247, 18)
(933891, 18)
(2287247,)
(933891,)


In [55]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(2287247, 19)
(933891, 19)


In [56]:
df_train = pd.DataFrame(train, columns=df_train.columns)
df_test = pd.DataFrame(test, columns=df_test.columns)

In [57]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

17869 17869
7296 7296


In [58]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [59]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [60]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(17869, 162)
(7296, 162)


In [61]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [62]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/X_train.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/X_test.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/y_train.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Two_Cows/y_test.csv", y_np_test, delimiter=",",fmt='%s')
     

In [63]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.95      0.98      0.97      2897
  not_eating       0.99      0.97      0.98      4399

    accuracy                           0.97      7296
   macro avg       0.97      0.98      0.97      7296
weighted avg       0.97      0.97      0.97      7296

